# K-Means Clustering of Toronto Neighborhoods based on Venue Similarity

This notebook clusters Toronto neighborhoods based on the similarity of the neighborhoods' venues. <br> Spatial information (i.e. venues) is fetched from Foursquare server via RESTful API calls. <br> Information on Toronto neighborhoods' geo-location is scraped from Wikipedia using the "beautiful soup" package.

## Scrape data from Wikipedia

##### I will use the "Beautiful soup" library to scrape a Wikipedia table containing information on Toronto neighborhoods 

In [3]:
# install beautiful soup
! pip install beautifulsoup4

##### First, I will get a local copy of the wikipedia article and save it in a html-file

In [6]:
import urllib.request

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
request = urllib.request.urlopen(url)
article = request.read().decode()

with open("List_of_postal_codes_of_Canada:_M.html", "w", encoding='utf-8') as html_source:
    html_source.write(article)

##### Second, I will load and parse the article with Beautiful soup. It will be saved in a text file

In [9]:
from bs4 import BeautifulSoup

# Load article, turn it into soup and get the <table>s
article = open("List_of_postal_codes_of_Canada:_M.html", encoding='utf-8').read()
soup = BeautifulSoup(article, "html.parser")
tables = soup.find_all("table", class_="sortable")

# Search through the tables for the one with the headings we want
for table in tables:
    table_headings = table.find_all("th")
    headings = [table_heading.text.strip() for table_heading in table_headings]
    if headings[:3] == ["Postal Code", "Borough", "Neighborhood"]:
        break

# Extract the columns and write to a semicolon-delimited text file
with open("List_of_postal_codes_of_Canada.txt", "w", encoding='utf-8') as text_file:
    for tr in table.find_all("tr"):
        tds = tr.find_all("td")
        if not tds:
            continue
        code, borough, neigh = [td.text.strip() for td in tds[:3]]
        
        print('; '.join([code, borough, neigh]), file=text_file)

## Load data into pandas dataframe and pre-process

##### I will load the data into a pandas dataframe

In [2]:
import pandas as pd

dataframe = pd.read_csv("List_of_postal_codes_of_Canada.txt", delimiter=";", header=None)
dataframe.columns = ["Postal Code", "Borough", "Neighborhood"]
dataframe.shape

(180, 3)

##### There are several lines for which no borough is assigned. These lines will be skipped

In [3]:
dataframe = dataframe[~dataframe["Borough"].str.contains("Not assigned")]
dataframe.reset_index(inplace=True, drop=True)
dataframe.shape

(103, 3)

##### Some postal codes may appear in more than one row with different neighborhood values. These rows will be combined into one row with the neighborhoods separated with a comma.

In [4]:
dataframe.groupby("Postal Code")["Neighborhood"].apply(", ".join)
print("Number of lines of the dataframe:", dataframe.shape[0])
print("Number of unique postal codes:", dataframe["Postal Code"].nunique())

Number of lines of the dataframe: 103
Number of unique postal codes: 103


##### Some rows have a value for feature "borough", but not for feature "neighborhood". In these cases, the "neighborhood" will be assigned the "borough"-value

In [5]:
dataframe["Neighborhood"][dataframe["Neighborhood"].str.contains("Not assigned")] = dataframe["Borough"]

##### Print the number of rows and columns of the frame

In [6]:
print(dataframe.shape)
dataframe.head()

(103, 3)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Geocode each postal code of Toronto

##### The geocoder python package will be used

In [34]:
! pip install geocoder

##### The following cell will separately write the latitude and longitude of each postal code into lists "latitudes" and "longitudes". <br> Since the geocoder API sometimes does respond, the program runs through a while-loop until all coordinates are found.  

In [ ]:
import geocoder # import geocoder

latitudes=[]
longitudes=[]

for postal_code in dataframe["Postal Code"]:
    # initialize variable to None
    lat_lng_coords = None
    # loop until the coordinates are found
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitudes.append(lat_lng_coords[0])
    longitudes.append(lat_lng_coords[1])


##### Since the geocoding duration is quite long, I have saved the coordinates in a csv-file

In [6]:
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### The dataframes now must be merged in order to obtain a final dataframe which can be used for clustering

In [12]:
toronto_frame = pd.merge(dataframe, coordinates, on=["Postal Code"])
toronto_frame.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Build clusters with K-Means and visualize the data

##### First, I will import libraries which I will use

In [10]:
import numpy as np # library for vectorized computation
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # for me, geopy is already installed
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# backend for rendering plots within the browser
%matplotlib inline 

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # for me, folium is already installed
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### 1. Create a map of Toronto with all neighborhoods

##### Use Geopy to obtain latitude and longitude of Toronto. I call my user agent "Toronto_explorer"

In [11]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto are 43.6534817, -79.3839347.


##### The following cell will render a map of Toronto including all neighborhoods. If you click on a marker on the map, a popup will show up indicating the neighborhood name and the borough name

In [15]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_frame["Latitude"], toronto_frame["Longitude"], toronto_frame["Borough"], toronto_frame["Neighborhood"]):
    label = "{}, {}".format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color="blue",
        fill=True,
        fill_color="#3186cc",
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 2. Fetch venue information for each postal code of Toronto using Foursquare API calls

##### Initialize Foursquare credentials

In [17]:
CLIENT_ID = 'AG3PTHAAGC2E5BJNTYTYFZ0024IBFGGT223CQ1RFRS1RL0CS' 
CLIENT_SECRET = 'SRA2OIUMNQ3B2JZSF5UPEA1Q2Z3ELKJVQ4PQIKAGPRSU05YF'
VERSION = '20180605' # Foursquare API version

##### Create a function that scrapes all venues for each postal code in a radius of 500 m from its centre from Foursquare

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=30):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    # create the dataframe of venues    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ["Neighborhoods", 
                  "Postal code Latitude", 
                  "Postal code Longitude", 
                  "Venue", 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

##### Call the function from above with Toronto data and save result in dataframe "toronto_venues"

In [28]:
toronto_venues = getNearbyVenues(names=toronto_frame["Neighborhood"],
                                   latitudes=toronto_frame["Latitude"],
                                   longitudes=toronto_frame["Longitude"]
                                  )

 Parkwoods
 Victoria Village
 Regent Park, Harbourfront
 Lawrence Manor, Lawrence Heights
 Queen's Park, Ontario Provincial Government
 Islington Avenue, Humber Valley Village
 Malvern, Rouge
 Don Mills
 Parkview Hill, Woodbine Gardens
 Garden District, Ryerson
 Glencairn
 West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
 Rouge Hill, Port Union, Highland Creek
 Don Mills
 Woodbine Heights
 St. James Town
 Humewood-Cedarvale
 Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
 Guildwood, Morningside, West Hill
 The Beaches
 Berczy Park
 Caledonia-Fairbanks
 Woburn
 Leaside
 Central Bay Street
 Christie
 Cedarbrae
 Hillcrest Village
 Bathurst Manor, Wilson Heights, Downsview North
 Thorncliffe Park
 Richmond, Adelaide, King
 Dufferin, Dovercourt Village
 Scarborough Village
 Fairview, Henry Farm, Oriole
 Northwood Park, York University
 East Toronto, Broadview North (Old East York)
 Harbourfront East, Union Station, Toronto Islands
 Little Portugal, Trini

##### Check the new dataframe

In [29]:
print(toronto_venues.shape)
toronto_venues.head()

(1337, 7)


,Neighborhoods,Postal code Latitude,Postal code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


##### Check how many venue categories were found in Toronto

In [30]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 233 unique categories.


### 3. Explore each postal code in detail

##### Create a dataframe that lists for each neighborhood whether the venue category can be found in the neighborhood or not

In [33]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhoods'] = toronto_venues['Neighborhoods'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhoods,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

##### Check the size of the new dataframe

In [34]:
toronto_onehot.shape

(1337, 234)

##### Group the neighborhoods/postal codes and calculate the mean of occurence of each category per neighborhood

In [98]:
toronto_grouped = toronto_onehot.groupby('Neighborhoods').mean().reset_index()
toronto_grouped.head()

,Neighborhoods,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,

##### Check the size of the new dataframe

In [99]:
toronto_grouped.shape

(95, 234)

##### Define a function that returns the most common venues per neighborhood

In [100]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:] #go through the category columns
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [101]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd'] # to get 1st, 2nd, 3rd as column headers

# create columns according to number of top venues
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind])) # top three most common venues
    except:
        columns.append('{}th Most Common Venue'.format(ind+1)) # fourth, fifth and higher most common venues

# create a new dataframe with column neighborhood and 10 most common venues
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Skating Rink,Clothing Store,Lounge,Breakfast Spot,Department Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
1,"Alderwood, Long Branch",Pizza Place,Sandwich Place,Coffee Shop,Skating Rink,Gym,Athletics & Sports,Pub,Yoga Studio,Department Store,Dessert Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Park,Mobile Phone Shop,Sandwich Place,Bridal Shop,Deli / Bodega,Restaurant,Ice Cream Shop,Supermarket
3,Bayview Village,Café,Japanese Restaurant,Chinese Restaurant,Bank,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Restaurant,Sandwich Place,Thai Restaurant,Pizza Place,Japanese Restaurant,Liquor Store,Indian Restaurant,Pub


### 3. Cluster the neighborhoods

##### Run k-Means to cluster the neighborhoods. Five clusters will do the job for me. I will use the grouped dataframe with the mean occurence of each venue category per neighborhood. <br> K-Means will cluster similar neighborhoods (i.e. similar occurence of same venues) into the same cluster.

In [102]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhoods', axis=1) # get rid of this feature as it is categorical

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 1, 1, 1, 1, 1, 1, 1, 1])

##### Create a dataframe including the cluster for each neighborhood

In [103]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)
neighborhoods_venues_sorted['Cluster Label'] = neighborhoods_venues_sorted['Cluster Label'].astype('int32')
toronto_merged = toronto_grouped.drop(toronto_grouped.columns[1:235], axis=1)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhoods'), on='Neighborhoods')
toronto_merged = toronto_merged.join(toronto_frame.set_index('Neighborhood'), on='Neighborhoods')

toronto_merged.head() #check columns to the right

,Neighborhoods,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postal Code,Borough,Latitude,Longitude
0,Agincourt,1,Latin American Restaurant,Skating Rink,Clothing Store,Lounge,Breakfast Spot,Department Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,M1S,Scarborough,43.794200,-79.262029
1,"Alderwood, Long Branch",4,Pizza Place,Sandwich Place,Coffee Shop,Skating Rink,Gym,Athletics & Sports,Pub,Yoga Studio,Department Store,Dessert Shop,M8W,Etobicoke,43.602414,-79.543484
2,"Bathurst Manor, Wilson Heights, Downsview North",1,Coffee Shop,Bank,Park,Mobile Phone Shop,Sandwich Place,Bridal Shop,Deli / Bodega,Restaurant,Ice Cream Shop,Supermarket,M3H,North York,43.754328,-79.442259
3,Bayview Village,1,Café,Japanese Restaurant,Chinese Restaurant,Bank,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,M2K,North York,43.786947,-79.385975
4,"Bedford Park, Lawrence Manor East",1,Italian Restaurant,Coffee Shop,Restaurant,Sandwich Place,Thai Restaurant,Pizza Place,Japanese Restaurant,Liquor Store,Indian Restaurant,Pub,M5M,North York,43.733283,-79.419750


##### Check number of neighborhoods per cluster

In [104]:
toronto_merged['Cluster Label'].value_counts()

1    79
3    12
4     6
2     1
0     1
Name: Cluster Label, dtype: int64

##### Finally, create a map to visualize the clusters

In [105]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11) # Toronto map

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neigh, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhoods'], toronto_merged['Cluster Label']):
    label = folium.Popup(str(neigh) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters